In [15]:
# Creating the connection
import pandas as pd
import numpy as np
# pip install python-binance
from binance.client import Client
import plotly.graph_objects as go
import telegram
import requests

# Binance API credentials
api_key = 'zMMoF47K9D6u1UrEs0SF5ZgwDtYsLfZYPPd2hDme5XtUJJOd6gogsVw8ibNu7mxM'
api_secret = 'xPqzQ87NvKFawOcYizIC81Ui3s7oQsBxPvXaD4t7LR85AtUhYeJL9XLnvwmoLPLN'

# Initialize Binance client
client = Client(api_key, api_secret)

#Telegram Bot
TOKEN = "7440240128:AAHGgBidb-mEjSOfzWXJ2hzY8UupUDlvKEs"
CHAT_ID = "5097888685"

In [ ]:
def get_historical_klines(symbol, interval, lookback):
    """
    Fetch historical klines (candlestick) data from Binance.

    :param symbol: Trading pair symbol (e.g., 'BTCUSDT')
    :param interval: Timeframe for candlesticks (e.g., '1h', '1d')
    :param lookback: Lookback period (e.g., '1 day ago UTC',2 months ago UTC)
    :return: Pandas DataFrame with OHLCV data
    """
    try:
        klines = client.get_historical_klines(symbol, interval, lookback)
        df = pd.DataFrame(klines, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume', 
            'close_time', 'quote_asset_volume', 'number_of_trades', 
            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
        ])
        df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        time_to_add = pd.Timedelta(hours=5, minutes=30)
        df['timestamp'] = df['timestamp']+time_to_add
        df.set_index('timestamp', inplace=True)
        df = df.astype(float)
        return df
    except Exception as e:
        raise Exception(f"Error fetching data: {e}")

In [ ]:
# Get Data
symbol = 'BTCUSDT'  # Trading pair symbol
interval = '5m'  # Time interval (e.g., '1h', '1d')
lookback = '1 day ago UTC'  # Lookback period

# Fetch data
df = get_historical_klines(symbol, interval, lookback)
df.to_csv('currentData.csv')
df

,open,high,low,close,volume
timestamp,,,,,
2025-02-16 19:25:00,96846.47,96888.62,96802.98,96842.37,59.52837
2025-02-16 19:30:00,96842.37,96842.38,96691.03,96778.58,83.58100
2025-02-16 19:35:00,96778.58,96878.48,96645.35,96878.48,108.62716
2025-02-16 19:40:00,96878.48,96940.00,96826.32,96848.45,33.91337
2025-02-16 19:45:00,96848.46,96848.46,96726.00,96765.64,23.70018
...,...,...,...,...,...
2025-02-17 19:00:00,96338.79,96347.60,96264.16,96271.47,19.82266
2025-02-17 19:05:00,96271.48,96325.52,96253.90,96325.51,15.80411
2025-02-17 19:10:00,96325.51,96356.52,96279.00,96356.52,30.98030


In [16]:
#Send telegram message
message = "hello, world"
url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={CHAT_ID}&text={message}"
r= requests.get(url)
print(r.json())

#Send telegram pictures
files = {'photo': open('alert/alert.PNG', 'rb')}
resp = requests.post('https://api.telegram.org/bot7440240128:AAHGgBidb-mEjSOfzWXJ2hzY8UupUDlvKEs/sendPhoto?chat_id=5097888685', files=files)
print(resp.json())

{'ok': True, 'result': {'message_id': 9, 'from': {'id': 7440240128, 'is_bot': True, 'first_name': 'Aladdin 2.0', 'username': 'Aladdin007_BOT'}, 'chat': {'id': 5097888685, 'first_name': 'Souvik', 'username': 'SouvikDaw', 'type': 'private'}, 'date': 1739804656, 'text': 'hello, world'}}
{'ok': True, 'result': {'message_id': 10, 'from': {'id': 7440240128, 'is_bot': True, 'first_name': 'Aladdin 2.0', 'username': 'Aladdin007_BOT'}, 'chat': {'id': 5097888685, 'first_name': 'Souvik', 'username': 'SouvikDaw', 'type': 'private'}, 'date': 1739804657, 'photo': [{'file_id': 'AgACAgUAAxkDAAMIZ7NPVYspqrzf_t7fiIaU8oQqEkMAAh7EMRu665hVMqmA_27tIM4BAAMCAANzAAM2BA', 'file_unique_id': 'AQADHsQxG7rrmFV4', 'file_size': 750, 'width': 90, 'height': 45}, {'file_id': 'AgACAgUAAxkDAAMIZ7NPVYspqrzf_t7fiIaU8oQqEkMAAh7EMRu665hVMqmA_27tIM4BAAMCAANtAAM2BA', 'file_unique_id': 'AQADHsQxG7rrmFVy', 'file_size': 11101, 'width': 320, 'height': 161}, {'file_id': 'AgACAgUAAxkDAAMIZ7NPVYspqrzf_t7fiIaU8oQqEkMAAh7EMRu665hVMqmA_27